In [ ]:
!mkdir ~/.kaggle                 #creates dir with kaggle directory
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Dataset Download
!kaggle datasets download -d ravirajsinh45/crop-and-weed-detection-data-with-bounding-boxes


# Data Extraction
import zipfile
zip_ref = zipfile.ZipFile('/content/crop-and-weed-detection-data-with-bounding-boxes.zip', 'r')     #ZipFile object points to downloaded zip file
zip_ref.extractall("/content/")
zip_ref.close()

In [ ]:
# Library Import
import os                           #interacts with os
import numpy as np                  #handle large and multidimensional arrays and matrices
import pandas as pd                 #manipulating numerical data
import matplotlib.pyplot as plt     #visualization

In [ ]:
info=pd.DataFrame(columns = ["Name","Class","X", "Y", "Width", "Height"])
print("Dataset Information\n",info)

In [ ]:
#empty list to store name, classification, x and y coordinates, width and height
name = []
clas = []
x = []
y = []
w = []
h = []

path = "/content/agri_data/data/"                           #directory path
for file in os.listdir(path):                               #loop iterates over each file
  if file.split(".")[-1] == "txt":                          #ensures only text files are processed
    with open(path + file, "r") as f :
      for line in f.readlines():
        data = line.split(" ")                              #index 0-class, 1-x, 2-y, 3-width, 4-height
        name.append(file.split(".")[0])
        clas.append(data[0])
        x.append(data[1])
        y.append(data[2])
        w.append(data[3])
        h.append(data[4])

In [ ]:
len(name), len(clas), len(x), len(y),len(w), len(h)     #gives number of elements

In [ ]:
# Saves and Prints information about the dataset
info["Name"] = pd.Series(name)
info["Class"] = pd.Series(clas)
info["X"] = pd.to_numeric(pd.Series(x))
info["Y"] = pd.to_numeric(pd.Series(y))
info["Width"] = pd.to_numeric(pd.Series(w))
info["Height"] = pd.to_numeric(pd.Series(h))

print("Information\n",info.info())

In [ ]:
# Creates directory named Cropped_data and two other directories Crop and Weed inside Cropped_data
!mkdir Cropped_data
!mkdir Cropped_data/Crop
!mkdir Cropped_data/Weed

In [ ]:
# IMAGE PROCESSING AND BOUNDING BOXES GENERATION 
# IMAGE CROP, RESIZE AND STORE SEPARATELY

from PIL import Image                                                     #PIL (Python Imaging Library) - for image processing

def crop_pic(image_name, x, y, w, h):                                     #function takes the name of image file and coordinates (x, y), width (w), and height (h) of the bounding box to be cropped
  source_path = "/content/agri_data/data/"
  image = plt.imread(f"{source_path}{image_name}.jpeg")
  #taking the shape
  W,H = image.shape[1],image.shape[0]
  #normalizing
  X = x*W
  Y = y*H
  width = w*W
  height = h*H

  x1 = int( X - (int(width) // 2))
  y1 = int( Y - (int(height) // 2))
  x2 = int( X + (int(width) // 2))
  y2 = int( Y + (int(height) // 2))
  print("Coordinates of bounding boxes",x1,x2, y1, y2)

  # Crop the image using the calculated coordinates
  cropped_image = image[y1:y2,  x1:x2]
  print("Coordinates of bounding boxes",W,H,x,y)
  return cropped_image

#Cropping the image and adding to its corresponding folder.
for index in range(info.shape[0]):                                        # through each row of data in a DataFrame info-contains information about the image and bounding boxes corrdinates
  cropped_pic = crop_pic( info.iloc[index,0], info.iloc[index,2], info.iloc[index,3], info.iloc[index,4],info.iloc[index,5] )
  reduced_img = Image.fromarray(cropped_pic)
  reduced_img = reduced_img.resize((256,256))                             #resize and cropped image
  print(info.iloc[index,1],info.iloc[index,0])
  if info.iloc[index,1] == '0':
    reduced_img.save(f"/content/Cropped_data/Crop/{index}.jpeg")
  else:
    reduced_img.save(f"/content/Cropped_data/Weed/{index}.jpeg")


print("Size of dataset",len(os.listdir("/content/Cropped_data/Crop"))+len(os.listdir("/content/Cropped_data/Weed")))
info.shape

In [ ]:
# LIBRARY FOR CNN MODEL GENERATION
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential

# DATASET (PROCESSED) INFORMATION
dataset = image_dataset_from_directory("/content/Cropped_data/",image_size = (256,256))
print("Class name; ",dataset.class_names)

In [ ]:
# Normalization
def process(image,label):
    image = tf.cast (image/255.0 ,tf.float32)
    return image,label
dataset = dataset.map(process)

In [ ]:
# create CNN model

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))


# MODEL SUMMARY
print("Model Summary\n",model.summary())

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history = model.fit(dataset, epochs=10, batch_size=32)

In [ ]:
# MODEL TESTING
test_image = plt.imread("/content/Cropped_data/Weed/1029.jpeg")
plt.imshow(test_image)
plt.show()
test_image = process(test_image,0)[0]
test_image = np.array(test_image).reshape((1,256,256,3))
# test_image = test_image.reshape((1,256,256,3))
if model.predict([test_image])[0]:
  print("Weed")
else:
  print("Crop")

In [ ]:
test_image = plt.imread("/content/Cropped_data/Crop/1034.jpeg")
plt.imshow(test_image)
plt.show()
test_image = process(test_image,0)[0]
test_image = np.array(test_image).reshape((1,256,256,3))
# model.predict([test_image])

if model.predict([test_image])[0]>0.50:
  print("Weed")
else:
  print("Crop")

In [ ]:
# PLOT FOR MODEL ACCURACY TESTING
plt.plot(history.history['accuracy'],label='train')
plt.legend()
plt.show()

In [ ]:
# ACCURACY CURVE PLOT
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# PLOT FOR LOSS CURVE
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()